# BIOS 8366 Final Project

Bob Chen, <bob.chen@vanderbilt.edu> (worked alone)

The dataset(s) used for the analyses described were obtained from Vanderbilt University Medical Center’s Synthetic Derivative which is supported by institutional funding and by the Vanderbilt CTSA grant ULTR000445 from NCATS/NIH

# Preprocessing 

## Import packages 

In [3]:
import csv
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from functools import reduce

## Load data files into memory

ADT and LAB had a couple issues importing but a few parameter changes to engine and quoting fixed it

In [4]:
adt=pd.read_csv("FONNESBECK_ADT_20151202.csv",engine='python')
lab=pd.read_csv("FONNESBECK_LAB_20151202.csv",engine='c',quoting=csv.QUOTE_NONE)
bmi=pd.read_csv("FONNESBECK_BMI_20151202.csv") 
bp=pd.read_csv("FONNESBECK_BP_20151202.csv")
cpt=pd.read_csv("FONNESBECK_CPT_20151202.csv")
egfr=pd.read_csv("FONNESBECK_EGFR_20151202.csv")
icd9=pd.read_csv("FONNESBECK_ICD9_20151202.csv")
med=pd.read_csv("FONNESBECK_MED_20151202.csv")
pheno=pd.read_csv("FONNESBECK_phenotype_20151202.csv")

## Preprocessing rationale

First I looked at the ADT dataset, as it contains the information that will become our target variable, the likelihood that a patient will undergo readmission within thirty days of being discharged. I interpreted missing data in the ADT datset as a limiting criteria, as without it, corresponding information from the other datasets become less useful as there could be no target variable information associated with it.

Next I took a look at the datasets which contained primarily quantitative, mostly numerical information that only needed slight encoding: BMI, BP, EGFR, and Pheno. I took some descriptive statistics for these data, with a few considerations for single entry RUIDs.

Then I examined datasets which contained categorical information such as CPT, ICD9, and MED. The encoding necessary for these datasets was far more intensive than the previous four, as the number of unique categories were upwards of a thousand. Still, I needed these data to be numerically encoded to use the modeling methods I had in mind.

In general, my preprocessing followed this general flow: 

1. Determine the types of information that would be useful as either a predictor or target variable. 
2. Drop irrelevant columns.
3. Drop rows with data missing information determined important in step 1
4. For each dataset and per RUID, generate descriptive statistics. 
5. Perform a merging of dataframes based on the common RUID as a key, this allows for a quick selection of only RUID's with the complete array of fefatures 
6. One-hot encode the remaining categorical variables and concatenate the resulting encodings to the comprehensive dataframe.
7. Write to file to import into another notebook etc.

## ADT preprocessing

It makes the most sense to get rid of columns with admission nulls first, as this is key information. We cannot know if this admission was within a 30 day time frame of a previous discharge if we do not know its date of occurrence. 

My goal in preprocessing the ADT data is primarily to transform admission and discharge dates into booleans and corresponding dates indicating instances of readmission within a 30 day time frame.  

In [5]:
admission_nulls=np.array(np.where(adt.Admission_date.isnull()))[0] #where are the missing admission dates?
discharge_nulls=np.array(np.where(adt.DISCHARGE_DATE.isnull()))[0] #where are the missing discharge dates?
adm_dis_nulls=set(admission_nulls)|set(discharge_nulls) #union of these missing dates
adm_dis_nulls=np.array(list(adm_dis_nulls)) 
adt_drop_adm_dis_nulls=adt.drop(adm_dis_nulls,axis=0) #drop the rows that contain these missing dates
remaining_nulls = np.where(adt_drop_adm_dis_nulls.isnull())[0] #check for remaining NaNs

In [6]:
adt_drop_adm_dis_nulls.iloc[remaining_nulls].head() #confirm that NaNs are in non-date columns

,RUID,Event,Admission_date,Event_Date,SRV_CODE,CHIEF_COMPLAINT,DISCHARGE_DATE
2162,52761375,Transfer,08/16/2007,08/16/2007,NUR,NaN,08/18/2007
2163,52761375,Admit,08/16/2007,08/16/2007,NUR,NaN,08/18/2007
2164,52761375,Discharge,08/16/2007,08/18/2007,NUR,NaN,08/18/2007
2349,52768576,Admit,05/30/2013,05/30/2013,NUR,NaN,06/01/2013
2350,52768576,Transfer,05/30/2013,05/30/2013,PED,NaN,06/01/2013


Still, all of these data need to be organised in a way such that we can easily do simple operations of admission and discharge dates to see if they fall within our 30 day interval of interest

In [7]:
#Drop columns with redundant information
discharge_rows=np.where(adt_drop_adm_dis_nulls.Event==('Discharge'))[0]
parsed_adt=adt_drop_adm_dis_nulls.iloc[discharge_rows]
parsed_adt=parsed_adt.drop(['Event','Event_Date','SRV_CODE','CHIEF_COMPLAINT'],axis=1)
parsed_adt=parsed_adt.sort_values(['RUID','Admission_date'],axis=0)
#Convert admission and discharge dates to a datetime type
parsed_adt.Admission_date=pd.to_datetime(parsed_adt.Admission_date)
parsed_adt.DISCHARGE_DATE=pd.to_datetime(parsed_adt.DISCHARGE_DATE)
parsed_adt=parsed_adt.sort_values(['RUID','Admission_date'])
#Set the RUID as the index
parsed_adt.index=parsed_adt.RUID
parsed_adt=parsed_adt.drop(['RUID'],axis=1)

In [8]:
parsed_adt.head()

,Admission_date,DISCHARGE_DATE
RUID,,
50135262,2007-02-08,2007-02-12
50135262,2007-08-03,2007-08-06
50135262,2007-08-28,2007-08-29
50135262,2008-02-24,2008-02-28
50135262,2008-04-12,2008-04-13


In [9]:
def readmission_check(adtIn):
    out_TTR=[]
    out_TTR_date=[]
    out_boolean=[]
    for ids in adtIn.index.unique():
        
        current_RUID=adtIn.loc[adtIn.index==ids]

        current_time_to_readmit=pd.DataFrame(np.array(current_RUID.Admission_date)[1:]-np.array(current_RUID.DISCHARGE_DATE)[:-1])
        current_time_to_readmit.columns=[ids]

        threshold_TTR=np.where(current_time_to_readmit<=pd.Timedelta('30 days'))[0]+1#[:,0]

        current_time_to_readmit_date=pd.DataFrame(np.array(current_RUID.iloc[threshold_TTR])[:,0])
        current_time_to_readmit_date.columns=[ids]

        out_TTR.append(current_time_to_readmit)
        out_TTR_date.append(current_time_to_readmit_date)
        
        readmit_within_threshold=(len(current_time_to_readmit_date)>0)
        out_boolean.append((ids,readmit_within_threshold))
    
    out_boolean=pd.DataFrame(out_boolean)
    out_boolean.columns=['RUID','Threshold Readmit']
    out_boolean.index=out_boolean.pop('RUID')
    
    return(out_TTR,out_TTR_date,out_boolean)

In [10]:
TTR,TTR_date,TTR_bool=readmission_check(parsed_adt)

In [11]:
TTR_bool.shape

(7889, 1)

All calculations were performed with respect to a single RUID at a time. Now we have usefully organized data regarding time to readmission, the date of these events, and a simple boolean of whether or not a patient experienced it.

## BMI preprocessing

BMI is the first of the quantitative datasets I looked at, and reasonably, many of the methods I applied here were also applied to the BP and EGFR.

In [12]:
bmi.head() #mean and std of bmi after qc, boolean of pregnancy

,RUID,BMI,Date_BMI,BMI_Weight,BMI_Height,Pregnancy_Indicator
0,50135262,41.43,01/09/2005,119.98,170.18,0
1,50135262,22.86,02/11/2011,70.00,175.00,0
2,50135262,43.07,02/12/2011,131.90,175.00,0
3,50135262,43.06,02/12/2011,131.86,175.00,0
4,50135262,41.47,02/12/2011,127.00,175.00,0


As a first pass of quality control, I found the top and bottom 2.5% of the values and dropped them. This is because the majority of the time, these were erroneous entries, for example, values such as -31 or 97600 are very unreasonable BMI values and are likely mistakes. Then, I removed rows that had null values.

In [13]:
bmi.BMI.describe(percentiles=[.025,.50,.975]) #drop the top and bottom 5% percentiles

count    262599.000000
mean         39.116793
std         589.110234
min         -31.160000
2.5%          9.240000
50%          25.240000
97.5%        49.140000
max       97600.000000
Name: BMI, dtype: float64

In [14]:
invalid_bmis_ind=np.array(list(set(np.where(bmi.BMI<9.24)[0])|set(np.where(bmi.BMI>49.14)[0])))
bmi_na_ind=np.where(bmi.BMI.isnull())[0]
drop_bmi=np.array(list(set(invalid_bmis_ind)|set(bmi_na_ind)))
bmi_qc=bmi.drop(drop_bmi,axis=0)

Here's a quick check to see that the numbers now fall into much more reasonable ranges, as extremely large and small numbers are now removed.

In [15]:
bmi_qc.BMI.describe()

count    249490.000000
mean         25.730957
std           8.241469
min           9.240000
25%          19.460000
50%          25.240000
75%          31.180000
max          49.140000
Name: BMI, dtype: float64

In [16]:
bmi_qc.index=bmi_qc.pop('RUID')
bmi_qc.head()

,BMI,Date_BMI,BMI_Weight,BMI_Height,Pregnancy_Indicator
RUID,,,,,
50135262,41.43,01/09/2005,119.98,170.18,0
50135262,22.86,02/11/2011,70.00,175.00,0
50135262,43.07,02/12/2011,131.90,175.00,0
50135262,43.06,02/12/2011,131.86,175.00,0
50135262,41.47,02/12/2011,127.00,175.00,0


In [17]:
def bmi_stats(bmiIn):
    
    out_bmi=[]
    
    for ids in bmiIn.index.unique():
        
        current_RUID=bmiIn.loc[bmiIn.index==ids]
        
        current_bmi_count=current_RUID.BMI.describe()[0]
        current_bmi_mean=current_RUID.BMI.describe()[1]
        current_bmi_std=current_RUID.BMI.std()
        current_bmi_min=current_RUID.BMI.describe()[2]
        current_bmi_25=current_RUID.BMI.describe()[3]
        current_bmi_50=current_RUID.BMI.describe()[4]
        current_bmi_75=current_RUID.BMI.describe()[5]
        current_bmi_max=current_RUID.BMI.describe()[6]
        
        #current_weight_count=current_RUID.BMI_weight.describe()[0]
        current_weight_mean=current_RUID.BMI_Weight.describe()[1]
        current_weight_std=current_RUID.BMI_Weight.std()
        current_weight_min=current_RUID.BMI_Weight.describe()[2]
        current_weight_25=current_RUID.BMI_Weight.describe()[3]
        current_weight_50=current_RUID.BMI_Weight.describe()[4]
        current_weight_75=current_RUID.BMI_Weight.describe()[5]
        current_weight_max=current_RUID.BMI_Weight.describe()[6]
        
        #current_height_count=current_RUID.BMI_height.describe()[0]
        current_height_mean=current_RUID.BMI_Height.describe()[1]
        current_height_std=current_RUID.BMI_Height.std()
        current_height_min=current_RUID.BMI_Height.describe()[2]
        current_height_25=current_RUID.BMI_Height.describe()[3]
        current_height_50=current_RUID.BMI_Height.describe()[4]
        current_height_75=current_RUID.BMI_Height.describe()[5]
        current_height_max=current_RUID.BMI_Height.describe()[6]
        
        if (current_bmi_count == 1):
            current_bmi_std=0
            current_bmi_min=current_bmi_50
            current_weight_std=0
            current_weight_min=current_weight_50
            current_height_std=0
            current_height_min=current_height_50
            
        current_preg=(current_RUID.Pregnancy_Indicator.sum()>0)
        
        current_list=list((ids,current_bmi_count,current_bmi_mean,current_bmi_std,current_bmi_min,current_bmi_25,current_bmi_50,current_bmi_75,current_bmi_max,
                           current_weight_mean,current_weight_std,current_weight_min,current_weight_25,current_weight_50,current_weight_75,current_weight_max,
                          current_height_mean,current_height_std,current_height_min,current_height_25,current_height_50,current_height_75,current_height_max,
                          current_preg))
        
        out_bmi.append(current_list)   
    
    out_bmi=pd.DataFrame(out_bmi)
    out_bmi.columns=['RUID','BMI Count','Mean BMI','BMI std','Min BMI','25% BMI','Med BMI','75% BMI','Max BMI',
                     'Mean Weight','Weight std','Min Weight','25% Weight','Med Weight','75% Weight','Max Weight',
                     'Mean Height','Height std','Min Height','25% Height','Med Height','75% Height','Max Height',
                     'Pregnancy']
    
    out_bmi.index=out_bmi.pop('RUID')
    
    return(out_bmi)

The descriptive statistics are mostly generated through the .describe() method, I noticed that often, calculated values such as Minimums and standard deviation would return NaN. This was primarily due to single entry RUIDs, which makes sense. To combat this issue, I recorded the total number of entries for the given measurements, which was one of the values returned by .describe(). This entry count was both useful as a patient feature as well as a quick check to decide wether or not to fill in the standard deviation (as 0) and minimum values (based on the median). 

In [18]:
bmi_stats_qc=bmi_stats(bmi_qc)

In [19]:
bmi_stats_qc.head()

,BMI Count,Mean BMI,BMI std,Min BMI,25% BMI,Med BMI,75% BMI,Max BMI,Mean Weight,Weight std,...,75% Weight,Max Weight,Mean Height,Height std,Min Height,25% Height,Med Height,75% Height,Max Height,Pregnancy
RUID,,,,,,,,,,,,,,,,,,,,,
50135262,70.0,43.069143,4.788758,4.788758,20.05,41.7525,44.005,45.4500,125.719286,13.161689,...,127.515,131.680,170.932571,2.303155,2.303155,160.00,170.18,170.18,170.2000,False
50135361,139.0,31.263237,3.733327,3.733327,13.26,27.8100,32.180,33.8650,105.721151,11.757472,...,105.730,115.475,184.033237,4.212298,4.212298,172.70,180.34,185.42,187.9600,False
50135369,112.0,26.232411,1.461247,1.461247,18.55,25.3475,26.305,27.2525,77.913482,4.256779,...,78.105,81.190,172.349464,1.357324,1.357324,165.10,172.72,172.72,172.7200,False
50135375,186.0,26.645968,2.160515,2.160515,23.13,24.7125,26.735,27.7350,77.133441,6.172237,...,77.355,80.240,170.152419,1.444665,1.444665,167.64,170.18,170.18,170.1800,False
50135425,94.0,32.783404,4.168860,4.168860,27.96,29.8875,31.695,34.0100,79.087553,9.267155,...,78.020,81.650,155.398936,1.328244,1.328244,152.40,154.94,154.94,155.8925,True


## BP preprocessing

The approach here was similar to BMI

In [20]:
bp.head() 

,RUID,SYSTOLIC,DIASTOLIC,Measure_date
0,50135262,150,80,01/09/2005
1,50135262,137,77,02/08/2007
2,50135262,137,78,02/08/2007
3,50135262,190,78,02/08/2007
4,50135262,190,77,02/08/2007


In [21]:
bp=bp.dropna(axis=0)

In [22]:
bp.SYSTOLIC.describe(percentiles=[0.025,0.5,0.975])

count    1.043040e+07
mean     1.189580e+02
std      2.702595e+01
min      0.000000e+00
2.5%     7.500000e+01
50%      1.170000e+02
97.5%    1.730000e+02
max      9.970000e+02
Name: SYSTOLIC, dtype: float64

In [23]:
bp.DIASTOLIC.describe(percentiles=[0.025,0.5,0.975])

count    1.043040e+07
mean     6.523043e+01
std      1.764321e+01
min      0.000000e+00
2.5%     3.800000e+01
50%      6.400000e+01
97.5%    9.800000e+01
max      9.760000e+02
Name: DIASTOLIC, dtype: float64

In [24]:
invalid_systolic=np.array(list(set(np.where(bp.SYSTOLIC<75)[0])|set(np.where(bp.SYSTOLIC>173)[0])))
invalid_diastolic=np.array(list(set(np.where(bp.DIASTOLIC<38)[0])|set(np.where(bp.DIASTOLIC>98)[0])))

In [25]:
drop_bp=np.array(list(set(invalid_systolic)|set(invalid_diastolic)))

In [26]:
bp=bp.drop(drop_bp,axis=0)

Again, I dropped the rows with unusually high or low values in the Systolic and Diastolic columns. This was in addition to rows with NA values.

In [27]:
bp.index=bp.pop('RUID')

In [28]:
bp.head()

,SYSTOLIC,DIASTOLIC,Measure_date
RUID,,,
50135262,150,80,01/09/2005
50135262,137,77,02/08/2007
50135262,137,78,02/08/2007
50135262,137,64,02/08/2007
50135262,160,72,02/09/2007


In [29]:
def bp_stats(bpIn):
    
    out_bp=[]
    
    for ids in bpIn.index.unique():
        
        current_RUID=bpIn.loc[bpIn.index==ids]
        
        current_systolic_count=current_RUID.SYSTOLIC.describe()[0]
        current_systolic_mean=current_RUID.SYSTOLIC.describe()[1]
        current_systolic_std=current_RUID.SYSTOLIC.std()
        current_systolic_min=current_RUID.SYSTOLIC.describe()[2]
        current_systolic_25=current_RUID.SYSTOLIC.describe()[3]
        current_systolic_50=current_RUID.SYSTOLIC.describe()[4]
        current_systolic_75=current_RUID.SYSTOLIC.describe()[5]
        current_systolic_max=current_RUID.SYSTOLIC.describe()[6]
                
        #current_diastolic_count=current_RUID.DIASTOLIC.describe()[0]
        current_diastolic_mean=current_RUID.DIASTOLIC.describe()[1]
        current_diastolic_std=current_RUID.DIASTOLIC.std()
        current_diastolic_min=current_RUID.DIASTOLIC.describe()[2]
        current_diastolic_25=current_RUID.DIASTOLIC.describe()[3]
        current_diastolic_50=current_RUID.DIASTOLIC.describe()[4]
        current_diastolic_75=current_RUID.DIASTOLIC.describe()[5]
        current_diastolic_max=current_RUID.DIASTOLIC.describe()[6]
        
        if (current_systolic_count == 1):
            current_systolic_std=0
            current_systolic_min=current_systolic_50
            current_diastolic_std=0
            current_diastolic_min=current_diastolic_50
            
        current_ratio=current_systolic_mean/current_diastolic_mean
        
        current_list=list((ids,current_systolic_count,current_systolic_mean,current_systolic_std,current_systolic_min,current_systolic_25,current_systolic_50,current_systolic_75,current_systolic_max,
                           current_diastolic_mean,current_diastolic_std,current_diastolic_min,current_diastolic_25,current_diastolic_50,current_diastolic_75,current_diastolic_max,current_ratio))
        
        out_bp.append(current_list)   
    
    out_bp=pd.DataFrame(out_bp)
    out_bp.columns=['RUID','BP Count','Mean Systolic','Systolic std','Min Systolic','25% Systolic','Med Systolic','75% Systolic','Max Systolic',
                    'Mean Diastolic','Diastolic std','Min Diastolic','25% Diastolic','Med Diastolic','75% Diastolic','Max Diastolic','S/D Ratio']
    out_bp.index=out_bp.pop('RUID')
    
    return(out_bp)

The approach here was very similar to BMI, relying heavily on the .describe() method for dataframes. However, there was the addition of the Systolic to Diastolic ratio value, though in the end this was a similar calculation to BMI.

In [30]:
bp_stats_qc=bp_stats(bp)

In [31]:
bp_stats_qc.head()

,BP Count,Mean Systolic,Systolic std,Min Systolic,25% Systolic,Med Systolic,75% Systolic,Max Systolic,Mean Diastolic,Diastolic std,Min Diastolic,25% Diastolic,Med Diastolic,75% Diastolic,Max Diastolic,S/D Ratio
RUID,,,,,,,,,,,,,,,,
50135262,3618.0,139.131564,18.461906,18.461906,85.0,127.0,141.0,154.0,62.709508,10.611751,10.611751,38.0,56.0,62.0,69.0,2.218668
50135361,5418.0,113.448874,17.444447,17.444447,75.0,101.0,113.0,125.0,60.080472,10.977034,10.977034,39.0,52.0,59.0,67.0,1.888282
50135369,4313.0,115.749362,17.429290,17.429290,75.0,103.0,116.0,127.0,69.808486,11.305381,11.305381,41.0,61.0,70.0,78.0,1.658099
50135375,2451.0,119.789882,18.563084,18.563084,77.0,105.0,122.0,133.0,68.225622,12.116277,12.116277,40.0,60.0,69.0,77.0,1.755790
50135425,622.0,127.321543,15.260048,15.260048,97.0,116.0,128.0,139.0,56.569132,8.982048,8.982048,41.0,50.0,55.0,61.0,2.250725


In [32]:
bp_stats_qc.shape

(7470, 16)

In [33]:
np.where(np.isnan(bp_stats_qc)) #final check for NaNs

(array([], dtype=int64), array([], dtype=int64))

## EGFR preprocessing

This quantitative dataset had the least diversity in information. Regardless, I used the same approach in processing EGFR as BP and BMI.

In [34]:
egfr.head() #mean std after qc

,RUID,EGFR,egfr_date
0,50135262,123.68000,02/08/2007
1,50135262,123.67783,02/08/2007
2,50135262,76.40173,02/11/2011
3,50135262,76.40000,02/11/2011
4,50135262,78.64000,02/12/2011


In [35]:
egfr=egfr.dropna(axis=0)

In [36]:
egfr.EGFR.describe(percentiles=[0.025,0.5,0.975])

count    323614.000000
mean        116.178002
std         153.138220
min           0.000000
2.5%          8.370000
50%          81.370000
97.5%       542.329680
max        3213.780000
Name: EGFR, dtype: float64

In [37]:
invalid_egfr_inds=np.array(list(set(np.where(egfr.EGFR<8.37)[0])|set(np.where(egfr.EGFR>542.32)[0])))

In [38]:
egfr=egfr.drop(invalid_egfr_inds,axis=0)

Again, I dropped the top and bottom 2.5% of values to eliminate outliers in addition to dropping NaN rows.

In [39]:
egfr.index=egfr.pop('RUID')

In [40]:
def egfr_stats(egfrIn):
    
    out_egfr=[]
    
    for ids in egfrIn.index.unique():
        
        current_RUID=egfrIn.loc[egfrIn.index==ids]
                
        current_egfr_count=current_RUID.EGFR.describe()[0]
        current_egfr_mean=current_RUID.EGFR.describe()[1]
        current_egfr_std=current_RUID.EGFR.std()
        current_egfr_min=current_RUID.EGFR.describe()[2]
        current_egfr_25=current_RUID.EGFR.describe()[3]
        current_egfr_50=current_RUID.EGFR.describe()[4]
        current_egfr_75=current_RUID.EGFR.describe()[5]
        current_egfr_max=current_RUID.EGFR.describe()[6]
        
        if (current_egfr_count == 1):
            current_egfr_std=0
            current_egfr_min=current_egfr_50
    
        current_list=list((ids,current_egfr_count,current_egfr_mean,current_egfr_std,current_egfr_min,current_egfr_25,current_egfr_50,current_egfr_75,current_egfr_max))
        
        out_egfr.append(current_list)   
    
    out_egfr=pd.DataFrame(out_egfr)
    out_egfr.columns=['RUID','EGFR Count','Mean EGFR','EGFR std','Min EGFR','25% EGFR','Med EGFR','75% EGFR','Max EGFR']
    out_egfr.index=out_egfr.pop('RUID')
    
    return(out_egfr)

This was the same approach as BMI and BP, but somewhat simpler as there was only one EGFR value per measurement instance.

In [41]:
egfr_stats_qc=egfr_stats(egfr)

In [42]:
egfr_stats_qc.head()

,EGFR Count,Mean EGFR,EGFR std,Min EGFR,25% EGFR,Med EGFR,75% EGFR,Max EGFR
RUID,,,,,,,,
50135262,136.0,87.336092,24.538705,24.538705,43.72504,70.083840,88.144675,104.009950
50135361,386.0,51.882929,18.630883,18.630883,13.64798,44.281087,52.662145,58.656810
50135369,226.0,84.715450,14.981761,14.981761,52.24956,75.487183,85.038710,93.732080
50135375,392.0,37.511314,18.827201,18.827201,8.89472,21.659688,36.244580,54.102325
50135425,50.0,46.618405,31.751456,31.751456,8.76485,24.017588,39.946965,50.207592


In [43]:
egfr_stats_qc.shape

(5893, 8)

## Phenotype preprocessing

For the most part this mixture of categorical and continuous values were simple to encode.  

In [45]:
pheno.head()

,RUID,Sex,DOB,DOD,Race
0,50135262,F,09/20/1949,NaN,W
1,50135361,M,02/15/1932,09/09/2012,W
2,50135369,M,05/04/1958,01/23/2012,W
3,50135375,M,05/01/1943,08/18/2011,B
4,50135425,F,10/02/1946,NaN,W


In [46]:
pheno.Race.value_counts()

W    5999
B    1193
U     341
H     315
A      94
N      50
I       8
Name: Race, dtype: int64

In [47]:
pheno.index=pheno.pop('RUID')
pheno['Female']=(pheno.Sex=='F')
pheno['Male']=(pheno.Sex=='M')
pheno.Female=pheno.Female.astype(int)
pheno.Male=pheno.Male.astype(int)

In [48]:
pheno.head()

,Sex,DOB,DOD,Race,Female,Male
RUID,,,,,,
50135262,F,09/20/1949,NaN,W,1,0
50135361,M,02/15/1932,09/09/2012,W,0,1
50135369,M,05/04/1958,01/23/2012,W,0,1
50135375,M,05/01/1943,08/18/2011,B,0,1
50135425,F,10/02/1946,NaN,W,1,0


In [49]:
pheno=pheno.drop(['Sex','DOD','Race'],axis=1)

Here, I opted to drop death day and race. This is because the medical records we are interested in all occur before this date. In addition, race was dropped. This was because the overwhelming majority (about 75%) of the entires were classified as white and some races were also very underrepresented. This overrepresentation may confound our modeling.

In [50]:
pheno.DOB=pd.to_datetime(pheno.DOB)

In [51]:
pheno=pheno.dropna(axis=0)

In [52]:
pheno.head()

,DOB,Female,Male
RUID,,,
50135262,1949-09-20,1,0
50135361,1932-02-15,0,1
50135369,1958-05-04,0,1
50135375,1943-05-01,0,1
50135425,1946-10-02,1,0


DOB had to be converted into datetime from string for calculations later on.

In [53]:
pheno.shape

(7957, 3)

## CPT

CPT was the first of the more complex categorical datasets. Key information that I extracted from this dataset as well as ICD9, Labs, and Meds was the count of each code.

In [54]:
cpt.head()

,RUID,CPT_Code,Event_date
0,50135262,00001,11/28/2004
1,50135262,00005,05/03/2012
2,50135262,00025,02/08/2007
3,50135262,00025,11/16/2004
4,50135262,00030,02/08/2007


In [55]:
cpt=cpt.dropna(axis=0)

In [56]:
cpt.index=cpt.pop('RUID')

In [57]:
cpt.head()

,CPT_Code,Event_date
RUID,,
50135262,00001,11/28/2004
50135262,00005,05/03/2012
50135262,00025,02/08/2007
50135262,00025,11/16/2004
50135262,00030,02/08/2007


In [58]:
def cpt_ranks(cptIn): 
    
    rank_out=[]
    
    for ids in cptIn.index.unique():
        
        current_RUID=cptIn.loc[cptIn.index==ids]
                
        top_cpt=list(current_RUID.CPT_Code.value_counts().index)
        
        top_cpt_count=list(current_RUID.CPT_Code.value_counts())[0]
        
        total_cpt_count=(current_RUID.CPT_Code.value_counts()).sum()
        
        current_list=list((ids,top_cpt[0],top_cpt_count,len(top_cpt),total_cpt_count))
        
        rank_out.append(current_list)   
    
    rank_out=pd.DataFrame(rank_out)
    rank_out.columns=['RUID','Top_CPT','Top_CPT_Count','Unique_CPTs','Total_CPT_Count']
    rank_out.index=rank_out.pop('RUID')
    
    return(rank_out)

For these categorical datasets, I primarily opted to take code counts. Through some examination, I observed that almost all RUID's had at least one code, but not all of them had more than one code. This somewhat limited the modeling I had planned on doing, as attempting to get count numbers and codes for the second, third, etc. most common entries returned NaNs or errors. Because of this, I stuck with just extracting information about the top codes as well as the number of unique and total code counts. I hypothesized that the number of these count values would be informative about the frequency at which patients had procedures or diagnoses occur.   

In [59]:
cpt_rankings=cpt_ranks(cpt)

In [60]:
cpt_rankings.head()

,Top_CPT,Top_CPT_Count,Unique_CPTs,Total_CPT_Count
RUID,,,,
50135262,82962,62,274,1130
50135361,85610,152,408,3126
50135369,94640,97,373,2277
50135375,36415,134,412,2872
50135425,80197,38,254,827


In [61]:
cpt_rankings.Top_CPT=cpt_rankings.Top_CPT.astype(str)

In [62]:
cpt_rankings.shape

(7891, 4)

In [63]:
np.where(cpt_rankings.isna())

(array([], dtype=int64), array([], dtype=int64))

## ICD9 preprocessing

I processed ICD9 codes with the same rationale as CPT, I thought about removing erroneous ICD9 codes but I figured they would be rare enough that it wouldn't make a huge difference.

In [64]:
icd9=icd9.dropna(axis=0)

In [65]:
icd9.head() #pull out most common codes or maybe first few 

,RUID,ICD9_Code,Event_date
0,50135262,00.40,08/04/2007
1,50135262,00.46,08/04/2007
2,50135262,00.66,08/04/2007
3,50135262,00.81,09/17/2012
4,50135262,00.82,09/17/2012


In [66]:
icd9.index=icd9.pop('RUID')

In [67]:
def icd9_ranks(icd9):
    
    rank_out=[]
    
    for ids in icd9.index.unique():
        
        current_RUID=icd9.loc[icd9.index==ids]
                
        top_icd9=list(current_RUID.ICD9_Code.value_counts().index)
        
        top_icd9_count=list(current_RUID.ICD9_Code.value_counts())[0]
        
        total_icd9_count=(current_RUID.ICD9_Code.value_counts()).sum()
        
        current_list=list((ids,top_icd9[0],top_icd9_count,len(top_icd9),total_icd9_count))
        
        rank_out.append(current_list)   
    
    rank_out=pd.DataFrame(rank_out)
    rank_out.columns=['RUID','Top_ICD9','Top_ICD9_Count','Unique_ICD9s','Total_ICD9_Count']
    rank_out.index=rank_out.pop('RUID')
    
    return(rank_out)

In [68]:
top_ranked_icd9s=icd9_ranks(icd9)

In [69]:
top_ranked_icd9s.head()

,Top_ICD9,Top_ICD9_Count,Unique_ICD9s,Total_ICD9_Count
RUID,,,,
50135262,250.00,46,211,705
50135361,496,108,356,2200
50135369,496,70,280,1142
50135375,V42.0,78,322,1662
50135425,V42.0,51,160,446


In [70]:
top_ranked_icd9s.shape

(7935, 4)

In [71]:
np.where(top_ranked_icd9s.isna())

(array([], dtype=int64), array([], dtype=int64))

## Labs preprocessing

The rationale here was similar to CPT and ICD9, I treated the Lab_name as a categorical code. I observed that the lab values were far too heterogeneous to go through and encode each one of them. I reasoned that the frequency of the lab test should be informative enough, as the repeated testing of some specific biological value would be enough to indicate that the patient was consistently being treated for something that had a defined battery of tests. 

In [72]:
lab.head() #maybe also pull out most common lab name due, because why would tests be done if there wasn't concern for them? values are far to heterogeneous

,RUID,Lab_name,Lab_date,Lab_value
0,50135262,%SAT,04/13/2008,17
1,50135262,ABO,02/08/2007,A
2,50135262,ABO,02/24/2008,a
3,50135262,AN-GAP,02/08/2007,9
4,50135262,AN-GAP,02/11/2011,8


In [73]:
lab=lab.dropna(axis=0)

In [74]:
lab.index=lab.pop('RUID')

In [75]:
lab.head()

,Lab_name,Lab_date,Lab_value
RUID,,,
50135262,%SAT,04/13/2008,17
50135262,ABO,02/08/2007,A
50135262,ABO,02/24/2008,a
50135262,AN-GAP,02/08/2007,9
50135262,AN-GAP,02/11/2011,8


In [76]:
def lab_ranks(labIn):
    
    rank_out=[]
    
    for ids in labIn.index.unique():
        
        current_RUID=labIn.loc[labIn.index==ids]
                
        top_lab=list(current_RUID.Lab_name.value_counts().index)
        
        top_lab_count=list(current_RUID.Lab_name.value_counts())[0]
        
        total_lab_count=(current_RUID.Lab_name.value_counts()).sum()

        current_list=list((ids,top_lab[0],top_lab_count,len(top_lab),total_lab_count))
        
        rank_out.append(current_list)   
    
    rank_out=pd.DataFrame(rank_out)
    rank_out.columns=['RUID','Top_Lab','Top_Lab_Count','Unique_Labs','Total_Lab_Count']
    rank_out.index=rank_out.pop('RUID')
    
    return(rank_out)

In [77]:
top_lab_ranks=lab_ranks(lab)

Again, the total lab count should be at least somewhat informative of some underlying medical issue, especially given the Top lab code. For example perhaps GluBed was some test measuring glucose levels for diabetics, its high count relative to the total lab count should be informative of some type of diabetic care the patient is undergoing.

In [78]:
top_lab_ranks.head()

,Top_Lab,Top_Lab_Count,Unique_Labs,Total_Lab_Count
RUID,,,,
50135262,GluBed,282,159,2345
50135361,PT-inr,311,270,6070
50135369,PCV,119,294,4530
50135375,GluBed,317,316,7790
50135425,Gluc,69,236,2520


## Meds preprocessing

This was a very heterogeneous dataset, especially with resspect to drug form, strength, route, dose, frerquency, and duration. The diversity in medications present made it difficult to safely use much of the information present. For example, even for specific drugs, their respective descriptive values were inconsistent in notation, making it extremely difficult to pull out relevant information, even for a single drug. I reasoned that again, the number of times that a drug would show up in the record was more reasonable to work with, just like the CPT and ICD9 codes. I hypothesized that certain drugs would only be consistently prescribed given the presence of some latent diagnosed issue or performed procedure. For example, opioid pain medication is likely to be prescribed after some type of invasive surgical procedure. 

In [79]:
med.head() #probably just pull out the x most common drug names

,RUID,Entry_Date,Drug_Name,DRUG_FORM,DRUG_STRENGTH,Route,Dose_Amt,Drug_Freq,Duration
0,50135262,02/13/2011,ACETAMINOPHEN : TYLENOL,NaN,NaN,PO,650.00000 MG,Q4H PRN,NaN
1,50135262,05/23/2012,ACETAMINOPHEN : TYLENOL,NaN,NaN,PO,650 MG,Q4H PRN,NaN
2,50135262,05/23/2012,ACETAMINOPHEN : TYLENOL,NaN,NaN,PO,650.00000 MG,Q4H PRN,NaN
3,50135262,10/29/2010,ACETAMINOPHEN : TYLENOL,NaN,NaN,PO,650 MG,ONCE,NaN
4,50135262,10/29/2010,ACETAMINOPHEN : TYLENOL,NaN,NaN,PO,650.00000 MG,ONCE,NaN


only interested in drugs they have been prescribed

In [80]:
med=med.drop(['DRUG_FORM','DRUG_STRENGTH','Route','Dose_Amt','Drug_Freq','Duration'],axis=1)

In [81]:
med=med.dropna(axis=0)

In [82]:
med.index=med.pop('RUID')

In [83]:
med.head()

,Entry_Date,Drug_Name
RUID,,
50135262,02/13/2011,ACETAMINOPHEN : TYLENOL
50135262,05/23/2012,ACETAMINOPHEN : TYLENOL
50135262,05/23/2012,ACETAMINOPHEN : TYLENOL
50135262,10/29/2010,ACETAMINOPHEN : TYLENOL
50135262,10/29/2010,ACETAMINOPHEN : TYLENOL


In [84]:
def med_ranks(medIn):
    
    rank_out=[]
    
    for ids in medIn.index.unique():
        
        current_RUID=medIn.loc[medIn.index==ids]
                
        top_med=list(current_RUID.Drug_Name.value_counts().index)
        
        top_med_count=list(current_RUID.Drug_Name.value_counts())[0]
        
        total_med_count=(current_RUID.Drug_Name.value_counts()).sum()

        current_list=list((ids,top_med[0],top_med_count,len(top_med),total_med_count))
        
        rank_out.append(current_list)   
    
    rank_out=pd.DataFrame(rank_out)
    rank_out.columns=['RUID','Top_Med','Top_Med_Count','Unique_Meds','Total_Med_Count']
    rank_out.index=rank_out.pop('RUID')
    
    return(rank_out)

In [85]:
top_med_ranks=med_ranks(med)

In [86]:
top_med_ranks.head()

,Top_Med,Top_Med_Count,Unique_Meds,Total_Med_Count
RUID,,,,
50135262,lantus,118,611,6177
50135361,amiodarone,602,750,15495
50135369,prednisone,470,750,10370
50135375,furosemide,339,762,14776
50135425,prednisone,185,370,2757


## Consolidation of the datasets based on common RUIDs

For further analysis, I had to consolidate these datasets into a single dataframe. This involes a number of steps regardless of the type of data:
    
1. Do a final check for the presence of NaNs
2. Do a final check for the proper value types
3. One-hot encode large amounts of unique categorical values
4. Do a join based on common RUIDs

In [87]:
TTR_bool['Threshold Readmit']=TTR_bool['Threshold Readmit'].astype(int)

In [88]:
TTR_bool.head()

,Threshold Readmit
RUID,
50135262,1
50135361,1
50135369,1
50135375,1
50135425,1


In [89]:
top_med_ranks.head()

,Top_Med,Top_Med_Count,Unique_Meds,Total_Med_Count
RUID,,,,
50135262,lantus,118,611,6177
50135361,amiodarone,602,750,15495
50135369,prednisone,470,750,10370
50135375,furosemide,339,762,14776
50135425,prednisone,185,370,2757


In [90]:
top_lab_ranks.head()

,Top_Lab,Top_Lab_Count,Unique_Labs,Total_Lab_Count
RUID,,,,
50135262,GluBed,282,159,2345
50135361,PT-inr,311,270,6070
50135369,PCV,119,294,4530
50135375,GluBed,317,316,7790
50135425,Gluc,69,236,2520


In [91]:
top_ranked_icd9s.head()

,Top_ICD9,Top_ICD9_Count,Unique_ICD9s,Total_ICD9_Count
RUID,,,,
50135262,250.00,46,211,705
50135361,496,108,356,2200
50135369,496,70,280,1142
50135375,V42.0,78,322,1662
50135425,V42.0,51,160,446


In [92]:
cpt_rankings.head()

,Top_CPT,Top_CPT_Count,Unique_CPTs,Total_CPT_Count
RUID,,,,
50135262,82962,62,274,1130
50135361,85610,152,408,3126
50135369,94640,97,373,2277
50135375,36415,134,412,2872
50135425,80197,38,254,827


In [93]:
pheno.head()

,DOB,Female,Male
RUID,,,
50135262,1949-09-20,1,0
50135361,1932-02-15,0,1
50135369,1958-05-04,0,1
50135375,1943-05-01,0,1
50135425,1946-10-02,1,0


In [94]:
egfr_stats_qc.head()

,EGFR Count,Mean EGFR,EGFR std,Min EGFR,25% EGFR,Med EGFR,75% EGFR,Max EGFR
RUID,,,,,,,,
50135262,136.0,87.336092,24.538705,24.538705,43.72504,70.083840,88.144675,104.009950
50135361,386.0,51.882929,18.630883,18.630883,13.64798,44.281087,52.662145,58.656810
50135369,226.0,84.715450,14.981761,14.981761,52.24956,75.487183,85.038710,93.732080
50135375,392.0,37.511314,18.827201,18.827201,8.89472,21.659688,36.244580,54.102325
50135425,50.0,46.618405,31.751456,31.751456,8.76485,24.017588,39.946965,50.207592


In [95]:
bp_stats_qc.head()

,BP Count,Mean Systolic,Systolic std,Min Systolic,25% Systolic,Med Systolic,75% Systolic,Max Systolic,Mean Diastolic,Diastolic std,Min Diastolic,25% Diastolic,Med Diastolic,75% Diastolic,Max Diastolic,S/D Ratio
RUID,,,,,,,,,,,,,,,,
50135262,3618.0,139.131564,18.461906,18.461906,85.0,127.0,141.0,154.0,62.709508,10.611751,10.611751,38.0,56.0,62.0,69.0,2.218668
50135361,5418.0,113.448874,17.444447,17.444447,75.0,101.0,113.0,125.0,60.080472,10.977034,10.977034,39.0,52.0,59.0,67.0,1.888282
50135369,4313.0,115.749362,17.429290,17.429290,75.0,103.0,116.0,127.0,69.808486,11.305381,11.305381,41.0,61.0,70.0,78.0,1.658099
50135375,2451.0,119.789882,18.563084,18.563084,77.0,105.0,122.0,133.0,68.225622,12.116277,12.116277,40.0,60.0,69.0,77.0,1.755790
50135425,622.0,127.321543,15.260048,15.260048,97.0,116.0,128.0,139.0,56.569132,8.982048,8.982048,41.0,50.0,55.0,61.0,2.250725


In [96]:
bmi_stats_qc.head()

,BMI Count,Mean BMI,BMI std,Min BMI,25% BMI,Med BMI,75% BMI,Max BMI,Mean Weight,Weight std,...,75% Weight,Max Weight,Mean Height,Height std,Min Height,25% Height,Med Height,75% Height,Max Height,Pregnancy
RUID,,,,,,,,,,,,,,,,,,,,,
50135262,70.0,43.069143,4.788758,4.788758,20.05,41.7525,44.005,45.4500,125.719286,13.161689,...,127.515,131.680,170.932571,2.303155,2.303155,160.00,170.18,170.18,170.2000,False
50135361,139.0,31.263237,3.733327,3.733327,13.26,27.8100,32.180,33.8650,105.721151,11.757472,...,105.730,115.475,184.033237,4.212298,4.212298,172.70,180.34,185.42,187.9600,False
50135369,112.0,26.232411,1.461247,1.461247,18.55,25.3475,26.305,27.2525,77.913482,4.256779,...,78.105,81.190,172.349464,1.357324,1.357324,165.10,172.72,172.72,172.7200,False
50135375,186.0,26.645968,2.160515,2.160515,23.13,24.7125,26.735,27.7350,77.133441,6.172237,...,77.355,80.240,170.152419,1.444665,1.444665,167.64,170.18,170.18,170.1800,False
50135425,94.0,32.783404,4.168860,4.168860,27.96,29.8875,31.695,34.0100,79.087553,9.267155,...,78.020,81.650,155.398936,1.328244,1.328244,152.40,154.94,154.94,155.8925,True


In [97]:
bmi_stats_qc.Pregnancy=bmi_stats_qc.Pregnancy.astype(int)

For convenience during testing, I saved the individual preprocessed datasets to disk as a checkpoint.

In [98]:
TTR_bool.to_csv("ADT_preprocessed_V3.csv")
bmi_stats_qc.to_csv("BMI_preprocessed_V3.csv")
pheno.to_csv("Pheno_preprocessed_V3.csv")
bp_stats_qc.to_csv("BP_preprocessed_V3.csv")
egfr_stats_qc.to_csv("EGFR_preprocessed_V3.csv")
cpt_rankings.to_csv("CPT_preprocessed_V3.csv")
top_ranked_icd9s.to_csv("ICD9_preprocessed_V3.csv")
top_lab_ranks.to_csv("LAB_preprocessed_V3.csv")
top_med_ranks.to_csv("MED_preprocessed_V3.csv")

## Merging datasets based on RUID

I used the reduce funtion from functools to merge all of these datasets based on a common RUID in the index.

In [99]:
complete_data=[TTR_bool,pheno,bmi_stats_qc,bp_stats_qc,egfr_stats_qc,cpt_rankings,top_ranked_icd9s,top_lab_ranks,top_med_ranks]

In [100]:
df_final = reduce(lambda left,right: pd.merge(left,right,on='RUID'), complete_data)

Did a bit more type checking and fixing 

In [101]:
df_final['Threshold Readmit']=df_final['Threshold Readmit'].astype(int)
df_final['Pregnancy']=df_final['Pregnancy'].astype(int)

In [102]:
df_final.head()

,Threshold Readmit,DOB,Female,Male,BMI Count,Mean BMI,BMI std,Min BMI,25% BMI,Med BMI,...,Unique_ICD9s,Total_ICD9_Count,Top_Lab,Top_Lab_Count,Unique_Labs,Total_Lab_Count,Top_Med,Top_Med_Count,Unique_Meds,Total_Med_Count
RUID,,,,,,,,,,,,,,,,,,,,,
50135262,1,1949-09-20,1,0,70.0,43.069143,4.788758,4.788758,20.05,41.7525,...,211,705,GluBed,282,159,2345,lantus,118,611,6177
50135361,1,1932-02-15,0,1,139.0,31.263237,3.733327,3.733327,13.26,27.8100,...,356,2200,PT-inr,311,270,6070,amiodarone,602,750,15495
50135369,1,1958-05-04,0,1,112.0,26.232411,1.461247,1.461247,18.55,25.3475,...,280,1142,PCV,119,294,4530,prednisone,470,750,10370
50135375,1,1943-05-01,0,1,186.0,26.645968,2.160515,2.160515,23.13,24.7125,...,322,1662,GluBed,317,316,7790,furosemide,339,762,14776
50135425,1,1946-10-02,1,0,94.0,32.783404,4.168860,4.168860,27.96,29.8875,...,160,446,Gluc,69,236,2520,prednisone,185,370,2757


## One-hot encoding categorical values

The number of unique categorical values present in the CPT, ICD9, Lab, and Med data required one hot encoding, as most implementations of the modeling methods I had in mind were incapable of processing purely categorical data in string form. These four datasets were one hot encoded in the same way, the unique values would exist as a column, and based on the individual RUID, the presence of that code (as a top or the most common code) in any of the categorical columns would recieve a binary value of 1 or 0.   

Encoding CPT codes

In [103]:
encoder_cpt = OneHotEncoder(handle_unknown='ignore')
enc_cpt=encoder_cpt.fit_transform(np.array(df_final.Top_ICD9).reshape(-1,1))
enc_cpt=pd.DataFrame(enc_cpt.todense())
enc_cpt.columns=(encoder_cpt.categories_)
enc_cpt=enc_cpt.astype(int)
enc_cpt=enc_cpt.add_prefix('CPT_Code_')

In [104]:
enc_cpt.head()

,CPT_Code_000.00,CPT_Code_041.4,CPT_Code_041.85,CPT_Code_042,CPT_Code_054.10,CPT_Code_070.54,CPT_Code_070.70,CPT_Code_075,CPT_Code_078.5,CPT_Code_079.99,...,CPT_Code_V66.7,CPT_Code_V67.00,CPT_Code_V67.09,CPT_Code_V70.0,CPT_Code_V70.7,CPT_Code_V72.31,CPT_Code_V74.5,CPT_Code_V76.12,CPT_Code_V76.2,CPT_Code_V76.51
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Encoding ICD9 codes

In [105]:
encoder_icd9 = OneHotEncoder(handle_unknown='ignore')
enc_icd9=encoder_icd9.fit_transform(np.array(df_final.Top_ICD9).reshape(-1,1))
enc_icd9=pd.DataFrame(enc_icd9.todense())
enc_icd9.columns=(encoder_icd9.categories_)
enc_icd9=enc_icd9.astype(int)
enc_icd9=enc_icd9.add_prefix('ICD9_Code_')

In [106]:
enc_icd9.head()

,ICD9_Code_000.00,ICD9_Code_041.4,ICD9_Code_041.85,ICD9_Code_042,ICD9_Code_054.10,ICD9_Code_070.54,ICD9_Code_070.70,ICD9_Code_075,ICD9_Code_078.5,ICD9_Code_079.99,...,ICD9_Code_V66.7,ICD9_Code_V67.00,ICD9_Code_V67.09,ICD9_Code_V70.0,ICD9_Code_V70.7,ICD9_Code_V72.31,ICD9_Code_V74.5,ICD9_Code_V76.12,ICD9_Code_V76.2,ICD9_Code_V76.51
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Encoding Lab codes

In [107]:
encoder_lab = OneHotEncoder(handle_unknown='ignore')
enc_lab=encoder_lab.fit_transform(np.array(df_final.Top_Lab).reshape(-1,1))
enc_lab=pd.DataFrame(enc_lab.todense())
enc_lab.columns=(encoder_lab.categories_)
enc_lab=enc_lab.astype(int)
enc_lab=enc_lab.add_prefix('Lab_Code_')

In [108]:
enc_lab.head()

,Lab_Code_17OHP,Lab_Code_ABO,Lab_Code_AN-GAP,Lab_Code_AcetMt,Lab_Code_Aceton,Lab_Code_Alb,Lab_Code_Aldo-P,Lab_Code_AlkP,Lab_Code_AmphCl,Lab_Code_BASORE,...,Lab_Code_WBC,Lab_Code_WBTrpI,Lab_Code_blStat,Lab_Code_blUnit,Lab_Code_pCO2,Lab_Code_pH,Lab_Code_pH-Ven,Lab_Code_pO2,Lab_Code_pO2-V,Lab_Code_rcSpO2
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [109]:
enc_lab.shape

(4797, 170)

Encoding Med codes

In [110]:
encoder_med = OneHotEncoder(handle_unknown='ignore')
enc_med=encoder_med.fit_transform(np.array(df_final.Top_Med).reshape(-1,1))
enc_med=pd.DataFrame(enc_med.todense())
enc_med.columns=(encoder_med.categories_)
enc_med=enc_med.astype(int)
enc_med=enc_med.add_prefix('Med_Code_')

In [111]:
enc_med.head()

,Med_Code_6 - mercaptopurine,Med_Code_ACETAMINOPHEN ORAL DROPS:,Med_Code_ACETYLCYSTEINE INJ,Med_Code_AGITATION SEDATION SCORE,Med_Code_ALPRAZOLAM: XANAX,Med_Code_ANAPHYLAXIS EMERGENCY KIT,Med_Code_ARIPIPRAZOLE: ABILIFY,Med_Code_ATENOLOL: TENORMIN,Med_Code_Abilify,Med_Code_Accu,...,Med_Code_zinc,Med_Code_ziprasidone,Med_Code_zocor,Med_Code_zofran,Med_Code_zoloft,Med_Code_zolpidem,Med_Code_zometa,Med_Code_zonisamide,Med_Code_zyprexa,Med_Code_zyrtec
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Concatenate all the newly encoded values, now we have about 3000 new columns of one hot encoded categorical values.

In [112]:
encoded=[enc_icd9,enc_lab,enc_med,enc_cpt]

In [113]:
encoded_values=pd.concat(encoded,axis=1)

In [114]:
encoded_values.shape

(4797, 2980)

In [115]:
encoded_values.head()

,ICD9_Code_000.00,ICD9_Code_041.4,ICD9_Code_041.85,ICD9_Code_042,ICD9_Code_054.10,ICD9_Code_070.54,ICD9_Code_070.70,ICD9_Code_075,ICD9_Code_078.5,ICD9_Code_079.99,...,CPT_Code_V66.7,CPT_Code_V67.00,CPT_Code_V67.09,CPT_Code_V70.0,CPT_Code_V70.7,CPT_Code_V72.31,CPT_Code_V74.5,CPT_Code_V76.12,CPT_Code_V76.2,CPT_Code_V76.51
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Assembling the final preprocessed dataset with encoded categorial values

All we need to do now is to concatenate the encoded categorical valeus to replace our Top_CPT, Top_ICD9, Top_Lab, and Top_Med columns. In addition, we also have to properly encode the DOB date as integers that can be understood by downstream analysis procedures.

In [116]:
df_final.head()

,Threshold Readmit,DOB,Female,Male,BMI Count,Mean BMI,BMI std,Min BMI,25% BMI,Med BMI,...,Unique_ICD9s,Total_ICD9_Count,Top_Lab,Top_Lab_Count,Unique_Labs,Total_Lab_Count,Top_Med,Top_Med_Count,Unique_Meds,Total_Med_Count
RUID,,,,,,,,,,,,,,,,,,,,,
50135262,1,1949-09-20,1,0,70.0,43.069143,4.788758,4.788758,20.05,41.7525,...,211,705,GluBed,282,159,2345,lantus,118,611,6177
50135361,1,1932-02-15,0,1,139.0,31.263237,3.733327,3.733327,13.26,27.8100,...,356,2200,PT-inr,311,270,6070,amiodarone,602,750,15495
50135369,1,1958-05-04,0,1,112.0,26.232411,1.461247,1.461247,18.55,25.3475,...,280,1142,PCV,119,294,4530,prednisone,470,750,10370
50135375,1,1943-05-01,0,1,186.0,26.645968,2.160515,2.160515,23.13,24.7125,...,322,1662,GluBed,317,316,7790,furosemide,339,762,14776
50135425,1,1946-10-02,1,0,94.0,32.783404,4.168860,4.168860,27.96,29.8875,...,160,446,Gluc,69,236,2520,prednisone,185,370,2757


In [117]:
df_final_dropped=df_final.drop(['Top_ICD9','Top_Lab','Top_Med','Top_CPT'],axis=1)

In [118]:
encoded_values.index=df_final.index

In [119]:
encoded_values.columns=encoded_values.columns.levels[0]

In [120]:
encoded_qc_data=pd.concat((df_final_dropped,encoded_values),axis=1)

In [121]:
encoded_qc_data.shape

(4797, 3043)

## Fixing value types and additional encoding for things like dates

In [122]:
dob=encoded_qc_data.DOB

In [123]:
dob=pd.to_datetime(dob)
Yob=dob.apply(lambda x: x.year)
Mob=dob.apply(lambda x: x.month)
Dob=dob.apply(lambda x: x.day)
encoded_qc_data['DoB']=Dob
encoded_qc_data['MoB']=Mob
encoded_qc_data['YoB']=Yob

In [124]:
encoded_qc_data=encoded_qc_data.drop(['DOB'],axis=1)

In [127]:
encoded_qc_data.head()

,Threshold Readmit,Female,Male,BMI Count,Mean BMI,BMI std,Min BMI,25% BMI,Med BMI,75% BMI,...,Med_Code_zofran,Med_Code_zoloft,Med_Code_zolpidem,Med_Code_zometa,Med_Code_zonisamide,Med_Code_zyprexa,Med_Code_zyrtec,DoB,MoB,YoB
RUID,,,,,,,,,,,,,,,,,,,,,
50135262,1,1,0,70.0,43.069143,4.788758,4.788758,20.05,41.7525,44.005,...,0,0,0,0,0,0,0,20,9,1949
50135361,1,0,1,139.0,31.263237,3.733327,3.733327,13.26,27.8100,32.180,...,0,0,0,0,0,0,0,15,2,1932
50135369,1,0,1,112.0,26.232411,1.461247,1.461247,18.55,25.3475,26.305,...,0,0,0,0,0,0,0,4,5,1958
50135375,1,0,1,186.0,26.645968,2.160515,2.160515,23.13,24.7125,26.735,...,0,0,0,0,0,0,0,1,5,1943
50135425,1,1,0,94.0,32.783404,4.168860,4.168860,27.96,29.8875,31.695,...,0,0,0,0,0,0,0,2,10,1946


In [126]:
np.where(encoded_qc_data.isna())

(array([], dtype=int64), array([], dtype=int64))

Our final, preprocessed dataset looks good to go, after a final check for any kind of weirdness that could result in NaNs being included

## Saving consolidated and preprocessed dataset to disk

In [223]:
encoded_qc_data.to_csv("Encoded_QC_Data_V3.csv")

In [224]:
pd.read_csv("Encoded_QC_Data_V3.csv",index_col=0)

,Threshold Readmit,Female,Male,BMI Count,Mean BMI,BMI std,Min BMI,25% BMI,Med BMI,75% BMI,...,Med_Code_zofran,Med_Code_zoloft,Med_Code_zolpidem,Med_Code_zometa,Med_Code_zonisamide,Med_Code_zyprexa,Med_Code_zyrtec,DoB,MoB,YoB
RUID,,,,,,,,,,,,,,,,,,,,,
50135262,1,1,0,70.0,43.069143,4.788758,4.788758,20.05,41.7525,44.005,...,0,0,0,0,0,0,0,20,9,1949
50135361,1,0,1,139.0,31.263237,3.733327,3.733327,13.26,27.8100,32.180,...,0,0,0,0,0,0,0,15,2,1932
50135369,1,0,1,112.0,26.232411,1.461247,1.461247,18.55,25.3475,26.305,...,0,0,0,0,0,0,0,4,5,1958
50135375,1,0,1,186.0,26.645968,2.160515,2.160515,23.13,24.7125,26.735,...,0,0,0,0,0,0,0,1,5,1943
50135425,1,1,0,94.0,32.783404,4.168860,4.168860,27.96,29.8875,31.695,...,0,0,0,0,0,0,0,2,10,1946
50135437,1,1,0,124.0,24.635726,2.383569,2.383569,19.16,23.1025,24.855,...,0,0,0,0,0,0,0,29,4,1969
50135624,1,0,1,171.0,23.018889,3.080134,3.080134,17.71,21.1050,22.280,...,0,0,0,0,0,0,0,28,11,1952
50135735,1,1,0,15.0,23.354667,1.928670,1.928670,21.58,21.7550,22.150,...,0,0,0,0,0,0,0,24,5,1973
50135759,0,0,1,85.0,30.601647,2.537609,2.537609,27.05,29.1900,30.580,...,0,0,0,0,0,0,0,5,2,1978
